<a href="https://colab.research.google.com/github/fffairforce/EECS738_Final_AutoML/blob/main/automl_model_2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install autokeras

     |████████████████████████████████| 174kB 2.8MB/s 
     |████████████████████████████████| 71kB 4.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=7f71219d1543cb23a627c669a434a531417970966608c3b9505bcf3ecaa60710
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=9fc0485bea571313532c8300179962f12cbc1b87e7a8ff78b243ae5e263a602b
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
pip install clearml

In [17]:
import numpy as np
import tensorflow as tf
import autokeras as ak
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from PIL import Image
from pathlib import Path
#preprocess.
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os  
import cv2 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


try clearml for visualizing callbacks in training/testing model results

In [15]:
from clearml import Task
task = Task.create(project_name='automl', task_name='automl_clf_model')
from tensorflow import keras


# Data Interpretation

This dataset contains 853 images belonging to the 3 classes, as well as their bounding boxes in the PASCAL VOC format.
The classes are:

With mask;
Without mask;
Mask worn incorrectly.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/drive/My Drive/EECS738/annotations'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Data prep
* label encoding

* train/test split

In [18]:
# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# print(x_train.shape)  # (60000, 28, 28)
# print(y_train.shape)  # (60000,)
# print(y_train[:3])  # array([7, 2, 1], dtype=uint8)
# Functions to load dataset
def load_image(img_file):
    """
    Loads image file, resize to 128 x 128 and return as numpy array
    :param img_file: path to image file in .jpg format
    :return: numpy array (128, 128, 3)
    """
    img = Image.open(img_file)
    img = img.resize((128, 128))
    img = np.array(img)
    return img

def load_dataset(data_dir, task='classification'):
    """
    Loads x and y dataset for classification or regression
    :param data_dir: path to data folder containing images and CSV file
    :param task: 'classification' or 'regression', default is 'classification'
    :return: x and y dataset
    """
    # loads x data, images sorted according to plot number
    img_list = [load_image(file) for file in sorted(Path(data_dir).glob('*.jpg'),key=lambda x: int(x.stem.split('_')[1]))]
    x = np.stack(img_list)
    # loads y data
    csv_file = Path(data_dir) / 'Lodging_data.csv'
    df = pd.read_csv(csv_file)
    if task == 'regression':
        df = df[df['Lodging'] == 'Yes']  # select rows for only lodged plots
        y = df['Score']
        x = [x[i] for i in df.index]  # slice x for only lodged plots
        x = np.array(x)
    else:
        y = df['Lodging']
    return x, y    

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
def assign_label(img,flower_type):
    return flower_type

X=[]
Z=[]
IMG_SIZE=150
FLOWER_DAISY_DIR='/content/drive/My Drive/EECS738/daisy/'
FLOWER_SUNFLOWER_DIR='/content/drive/My Drive/EECS738/sunflower/'
FLOWER_TULIP_DIR='/content/drive/My Drive/EECS738/tulip'
FLOWER_DANDI_DIR='/content/drive/My Drive/EECS738/dandelion'
FLOWER_ROSE_DIR='/content/drive/My Drive/EECS738/rose'
def assign_label(img,flower_type):
    return flower_type

def make_train_data(flower_type,DIR):
    for img in tqdm(os.listdir(DIR)):
        label=assign_label(img,flower_type)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
        X.append(np.array(img))
        Z.append(str(label))    

In [19]:
make_train_data('Daisy',FLOWER_DAISY_DIR)
make_train_data('Sunflower',FLOWER_SUNFLOWER_DIR)
make_train_data('Tulip',FLOWER_TULIP_DIR)
make_train_data('Dandelion',FLOWER_DANDI_DIR)
make_train_data('Rose',FLOWER_ROSE_DIR)

100%|██████████| 784/784 [02:38<00:00,  4.95it/s]


In [ ]:
 # %reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [22]:
# debug block
print(x_train.shape)
# print(y_train[:10])
print(X.shape)
# print(x_train[:1])
# Z

(2896, 150, 150, 3)
(4323, 150, 150, 3)


In [20]:
from keras.utils import to_categorical
le=LabelEncoder()
Y=le.fit_transform(Z)
Y=to_categorical(Y,5)
X=np.array(X)
# X=X/255
# split train/test
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.33,random_state=42)


apply autokeras classification

In [ ]:
# Initialize the image classifier.
clf = ak.ImageClassifier(overwrite=True, max_trials=1)
# Feed the image classifier with training data.
clf.fit(x_train, y_train, epochs=10)


# Predict with the best model.
predicted_y = clf.predict(x_test)
print(predicted_y)


# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

NameError: ignored

explore classification models in autokeras

3classic models to look at:

CNN, ResNet, Xception

In [ ]:

input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node1 = ak.ConvBlock()(output_node)
output_node2 = ak.ResNetBlock(version="v2")(output_node)
output_node3 = ak.XceptionBlock()(output_node)
# output_node3 = ak.DenseBlock()(output_node)
# output_node4 = ak.Embedding()(output_node)

output_node = ak.Merge()([output_node1, output_node2, output_node3])
output_node = ak.ClassificationHead()(output_node)

auto_model = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)

# clf = autokeras.ImageClassifier(
#     num_classes=None,
#     multi_label=False,
#     loss=None,
#     metrics=None,
#     project_name="image_classifier",
#     max_trials=100,
#     directory=None,
#     objective="val_loss",
#     tuner=None,
#     overwrite=False,
#     seed=42,
#     max_model_size=None,
#     **kwargs
# )
tensorboard_callback_train = keras.callbacks.TensorBoard(log_dir='log',histogram_freq=1,
  write_images=True)
# Feed the AutoModel with training data.
auto_model.fit(x_train[:], y_train[:], epochs=10, callbacks=[tensorboard_callback_train])
# Predict with the best model.
predicted_y = auto_model.predict(x_test)

# Evaluate the best model with testing data.
print(auto_model.evaluate(x_test, y_test))
# get the best performing model
model = auto_model.export_model()
# summarize the loaded model
model.summary()
# save the best performing model to file
model.save('model_clf.h5')
# save results summary
auto_model.tuner.results_summary()

Trial 1 Complete [05h 14m 35s]
val_loss: 1.4878095388412476

Best val_loss So Far: 1.4878095388412476
Total elapsed time: 05h 14m 35s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
72/91 [======================>.......] - ETA: 7:34 - loss: 7.1963 - accuracy: 0.3547

Customized Search Space for autokeras 

In [ ]:
# model = ak.AutoModel(tuner='bayesian',
#                      inputs=input_node,
#                      outputs=output_node,
#                      max_trials=100,
#                      overwrite=True,
#                      seed=10)

# # Train model
# model.fit(x_train, y_train, epochs=200,
#           validation_data=(x_val, y_val))    

# Evaluate model
# score = model.evaluate(x_test, y_test)

input_node = ak.ImageInput()
output_node = ak.ImageBlock()(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(x_train, y_train, epochs=100)

Trial 1 Complete [00h 14m 53s]
val_loss: 0.4090674817562103

Best val_loss So Far: 0.4090674817562103
Total elapsed time: 00h 14m 53s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
36/36 [==============================] - 100s 3s/step - loss: 0.8529 - accuracy: 0.4892
Epoch 2/10
36/36 [==============================] - 97s 3s/step - loss: 0.8187 - accuracy: 0.5911
Epoch 3/10
36/36 [==============================] - 97s 3s/step - loss: 0.5810 - accuracy: 0.6614
Epoch 4/10
36/36 [==============================] - 97s 3s/step - loss: 0.5181 - accuracy: 0.7177
Epoch 5/10
36/36 [==============================] - 96s 3s/step - loss: 0.4861 - accuracy: 0.7423
Epoch 6/10
36/36 [==============================] - 96s 3s/step - loss: 0.4591 - accuracy: 0.7592
Epoch 7/10
36/36 [==============================] - 96s 3s/step - loss: 0.4379 - accuracy: 0.7727
Epoch 8/10
36/36 [==============================] - 98s 3s/step - loss: 0.4205 - accuracy: 0.7879
Epoch 9/10
36/36 [=========================

In [ ]:
# Evaluate model
score = clf.evaluate(x_test, y_test)

Visualize autokeras model

In [ ]:
from trains import Task

task = Task.init(project_name="autokeras", task_name="autokeras imdb example with scalars")

Try DIY automl based on sklearn model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_selector

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC,SVC
from sklearn.datasets import load_wine
from sklearn.metrics import balanced_accuracy_score

notes:

*   The purpose of the pipeline is to assemble several steps that can be cross-validated together while setting different parameters.

*   sklearn.pipeline.Pipeline(steps, *, memory=None, verbose=False);steps=[List of (name, transform) tuples (implementing fit/transform) that are chained]

*   ColumnTransformer: Applies transformers to columns of an array or pandas DataFrame.
*   RandomizedSearchCV(estimator, param_distributions, *, n_iter=10, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', random_state=None, error_score=nan, return_train_score=False): param_distributions = optimization_grid = dict or list of dicts




In [ ]:
class AutoMLClassifier:
  def __init__(self, scoring_fuction = 'balanced_accuracy', n_iter = 50):
    self.scoring_fuction = scoring_fuction
    self.n_iter = n_iter
  
  def fit(self, X, y):
    X_train = X
    y_train = y

    # detect distinct value of categorical variables
    categorical_values = []
    cat_subset = X_train.select_dtypes(include = ['object','category','bool'])#e.g. for pd_dataframe
    #cat_subset = y_train #change in terms of labeling in image dataset
    for i in range(cat_subset.shape[1]):
      categorical_values.append(list(cat_subset.iloc[:,i].dropna().unique()))
    #categorical_values = y_train # for this demo data it's concated in y_train matrix

    # defining a pre-processing pipeline
    num_pipeline = Pipeline([
      ('cleaner',SimpleImputer()),
      ('scaler',StandardScaler())
    ])

    cat_pipeline = Pipeline([
      ('cleaner',SimpleImputer(strategy = 'most_frequent')),
      ('encoder',OneHotEncoder(sparse = False, categories=categorical_values))
    ])


    preprocessor = ColumnTransformer([
      ('numerical', num_pipeline, make_column_selector(dtype_exclude=['object','category','bool'])),
      ('categorical', cat_pipeline, make_column_selector(dtype_include=['object','category','bool']))
    ])#need to be transfered for image data

    model_pipeline_steps = []
    model_pipeline_steps.append(('preprocessor',preprocessor))
    model_pipeline_steps.append(('feature_selector',SelectKBest(f_classif,k='all'))) # “all” option bypasses selection, for use in a parameter search.
    model_pipeline_steps.append(('estimator',LogisticRegression()))
    model_pipeline = Pipeline(model_pipeline_steps)

    total_features = preprocessor.fit_transform(X_train).shape[1]

    # set randam search optimizers
    optimization_grid = []
    # KNN
    optimization_grid.append({
        'preprocessor_numerical_scaler':[RobustScaler(),StandardScaler(),MinMaxScaler()],
        'preprocessor_numerical_cleaner_strategy':['mean','median'],
        'feature_selector_k':list(np.arange(1,total_features,5))+['all'],
        'estimator':[KNeighborsClassifier()],
        'estimator__weights':['uniform','distance'],
        'estimator__n_neighbors':np.arange(1,20,1)
    })

    # Random forest
    optimization_grid.append({
        'preprocessor__numerical__scaler':[None],
        'preprocessor__numerical__cleaner__strategy':['mean','median'],
        'feature_selector__k': list(np.arange(1,total_features,5)) + ['all'],
        'estimator':[RandomForestClassifier(random_state=0)],
        'estimator__n_estimators':np.arange(5,500,10),
        'estimator__criterion':['gini','entropy']
    })

    # Gradient boosting
    optimization_grid.append({
        'preprocessor__numerical__scaler':[None],
        'preprocessor__numerical__cleaner__strategy':['mean','median'],
        'feature_selector__k': list(np.arange(1,total_features,5)) + ['all'],
        'estimator':[GradientBoostingClassifier(random_state=0)],
        'estimator__n_estimators':np.arange(5,500,10),
        'estimator__learning_rate':np.linspace(0.1,0.9,20),
    })

    # Linear SVM
    optimization_grid.append({
        'preprocessor__numerical__scaler':[RobustScaler(),StandardScaler(),MinMaxScaler()],
        'preprocessor__numerical__cleaner__strategy':['mean','median'],
        'feature_selector__k': list(np.arange(1,total_features,5)) + ['all'],
        'estimator':[LinearSVC(random_state = 0)],
        'estimator__C': np.arange(0.1,1,0.1),
        
    }) 
    
    # Logistic regression
    optimization_grid.append({
        'preprocessor__numerical__scaler':[RobustScaler(),StandardScaler(),MinMaxScaler()],
        'preprocessor__numerical__cleaner__strategy':['mean','median'],
        'feature_selector__k': list(np.arange(1,total_features,5)) + ['all'],
        'estimator':[LogisticRegression()]
    })

    # operate randam search
    search = RandomizedSearchCV(
      model_pipeline,
      optimization_grid,
      n_iter=self.n_iter,
      scoring = self.scoring_function, 
      n_jobs = -1, 
      random_state = 0, 
      verbose = 3,
      cv = 5 #5-fold cross validation
    )
    search.fit(X_train.y_train)
    self.best_estimator = search.best_estimator_
    self.best_pipeline = search.best_params_

  def naive_fit(self,X,y):
    X_train = X
    y_train = y
    

  def predict(self,X,y=None):
    return self.best_estimator.predict(X)

  def predict_prob(self,X,y=None):
    return self.best_estimator.predict_proba(X)

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
# detect distinct value of categorical variables
categorical_values = []
cat_subset = X_train.select_dtypes(include = ['object','category','bool'])#e.g. for pd_dataframe
#cat_subset = y_train #change in terms of labeling in image dataset
for i in range(cat_subset.shape[1]):
  categorical_values.append(list(cat_subset.iloc[:,i].dropna().unique()))
#categorical_values = y_train # for this demo data it's concated in y_train matrix

# defining a pre-processing pipeline
num_pipeline = Pipeline([
  ('cleaner',SimpleImputer()),
  ('scaler',StandardScaler())
])

cat_pipeline = Pipeline([
  ('cleaner',SimpleImputer(strategy = 'most_frequent')),
  ('encoder',OneHotEncoder(sparse = False, categories=categorical_values))
])


preprocessor = ColumnTransformer([
  ('numerical', num_pipeline, make_column_selector(dtype_exclude=['object','category','bool'])),
  ('categorical', cat_pipeline, make_column_selector(dtype_include=['object','category','bool']))
])#need to be transfered for image data

model_pipeline_steps = []
model_pipeline_steps.append(('preprocessor',preprocessor))
model_pipeline_steps.append(('feature_selector',SelectKBest(f_classif,k='all'))) # “all” option bypasses selection, for use in a parameter search.
model_pipeline_steps.append(('estimator',LogisticRegression()))
model_pipeline = Pipeline(model_pipeline_steps)

total_features = preprocessor.fit_transform(X_train).shape[1]
print(total_features)
# set randam search optimizers
optimization_grid = []
# KNN
optimization_grid.append({
    'preprocessor_numerical_scaler':[RobustScaler(),StandardScaler(),MinMaxScaler()],
    'preprocessor_numerical_cleaner_strategy':['mean','median'],
    'feature_selector_k':list(np.arange(1,total_features,5))+['all'],
    'estimator':[KNeighborsClassifier()],
    'estimator__weights':['uniform','distance'],
    'estimator__n_neighbors':np.arange(1,20,1)
        })
print(optimization_grid)

13
[{'preprocessor_numerical_scaler': [RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
             with_scaling=True), StandardScaler(copy=True, with_mean=True, with_std=True), MinMaxScaler(copy=True, feature_range=(0, 1))], 'preprocessor_numerical_cleaner_strategy': ['mean', 'median'], 'feature_selector_k': [1, 6, 11, 'all'], 'estimator': [KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')], 'estimator__weights': ['uniform', 'distance'], 'estimator__n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])}]


try 

In [ ]:
# d = load_wine()
# y = d['target']
# X = pd.DataFrame(d['data'],columns = d['feature_names'])

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

model = AutoMLClassifier()
model.fit(X_train,y_train)

AttributeError: ignored

In [ ]:
X.shape

NameError: ignored

In [ ]:
categorical_values=y_train
OneHotEncoder(sparse = False, categories=categorical_values)
total_features = preprocessor.fit_transform(X_train).shape[1]

NameError: ignored

data load

In [ ]:
# d = load_data()
#labeling
# y = d['label']# adjust for needed
# X = data_input_transform# adjust for needed
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=10)

# use x_train, y_train(label) from previous test
model = AutoMLClassifier()
model.fit(x_train,y_train)

ValueError: ignored

The balanced accuracy in binary and multiclass classification problems to deal with imbalanced datasets. It is defined as the average of recall obtained on each class

In [ ]:
balanced_accuracy_score(y_test, model.predict(X_test))

fit model, see which model and preprocessing parameters have been chosen

In [ ]:
model.best_pipeline